In [ ]:
import numpy as np
import cv2

from foundry import Foundry
from garden_ai import GardenClient, Model, step
from garden_ai.mlmodel import LocalModel

In [ ]:
# instantiate the Garden client
client = GardenClient()

In [ ]:
# convert our saved model into a Garden LocalModel
local_model = LocalModel(model_name="dendrite_segmentation",
                         flavor="tensorflow",
                         local_path="model.h5",
                         user_email=client.get_email())

In [ ]:
# register the LocalModel with Garden using the client
registered_model = client.register_model(local_model)

In [ ]:
# Garden's pipelines are composed of steps, so let's make one to run the model
@step
def run_inference(input_arg: object, model=Model(registered_model.full_name)) -> object:
    return model.predict(input_arg)

In [ ]:
# add the step to a pipeline
pipeline = client.create_pipeline(["Isaac Darling"],
                                  "Dendrite Segmentation Test Pipeline",
                                  short_name=registered_model.model_name,
                                  steps=(run_inference,),
                                  pip_dependencies=["tensorflow"],
                                  description="Semantic dendrite segmentation via machine learning",
                                  tags=["materials science",
                                        "x-ray",
                                        "segmentation",
                                        "computer vision"])

In [ ]:
# register the pipeline with Garden!
registered = client.register_pipeline(pipeline)
registered

In [ ]:
# create a garden and add the pipeline to it, then publish it to the search index
garden = client.create_garden(registered.authors,
                              registered.title,
                              pipelines=[registered],
                              pipeline_ids=[registered.doi],
                              description=registered.description,
                              tags=registered.tags)

client.publish_garden_metadata(garden)

In [ ]:
# confirm garden is searchable on the index
print(garden == client.get_published_garden(garden.doi))
garden

In [ ]:
# load in foundry data
f = Foundry(index="mdf", no_local_server=False, no_browser=False)
f.load("foundry_stan_segmentation_v1.1", globus=True)

res = f.load_data()
X = res["train"]["input"]["train/input"]

new_size = 32*10
X_r = np.array([cv2.resize(img, dsize=(new_size, new_size))
     for img in X])

In [ ]:
# run the pipeline!
garden.dendrite_segmentation(X_r)